# Método Heurístico

In [7]:
# Print into txt file...
import sys
sys.stdout = open("output-solution.txt", "w", encoding="utf-8")

In [8]:
# Código completo con inventario formateado como DataFrame e impreso junto al resto:

import pandas as pd
from scipy.spatial.distance import euclidean

# === PARÁMETROS ===
dias_totales = 6
aclimatacion_min_dias = 1

# Coordenadas de polígonos (18 = almacén)
poligonos_coords = {
    1: (0, 0), 2: (40, 20), 18: (30, 60)
}
almacen_coord = poligonos_coords[1]

# Demanda por especie y polígono
demanda_poligonos = {
    "Encino": {
        1: 10, 2: 15
    }
}

# Proveedores
demandas_oferta = {
    "Prov1": {"Encino": {"costo": 5, "max_oferta": 12}},
    "Prov2": {"Encino": {"costo": 10, "max_oferta": 30}}
}

# Logística
capacidad_camion = 8
costo_transporte = 200
velocidad = 60            # km/h
tiempo_carga = 1         # minutos
tiempo_descarga = 1      # minutos
jornada_min = 300         # minutos
espacio_max_almacen = 15
costo_plantacion = 1

def tiempo_entre(p1, p2):
    distancia = euclidean(poligonos_coords[p1], poligonos_coords[p2])
    return (distancia / velocidad) * 60

def actualizar_inventario(inventario, dia):
    if dia == 0:
        return
    for esp in inventario:
        inventario[esp][dia] += inventario[esp][dia - 1]

def calcular_disponibles(inventario):
    disponibles = {esp: [0] * (dias_totales + 1) for esp in demanda_poligonos}
    for dia in range(dias_totales + 1):
        for esp in demanda_poligonos:
            if dia >= aclimatacion_min_dias:
                disponibles[esp][dia] = inventario[esp][dia - aclimatacion_min_dias]
    return disponibles

def planificar_rutas(dia, disponibles, demanda_restante):
    rutas_dia = []
    entregado = {esp: 0 for esp in demanda_poligonos}
    tiempo_total = 0

    while tiempo_total < jornada_min:
        ruta = [18]
        tiempo_ruta = tiempo_carga
        carga = 0
        detalle = []

        while True:
            last = ruta[-1]
            candidatos = [pid for esp in demanda_restante for pid, d in demanda_restante[esp].items() if d > 0]
            candidatos.sort(key=lambda pid: euclidean(poligonos_coords[last], poligonos_coords[pid]))
            encontrado = False

            for pid in candidatos:
                t_viaje = tiempo_entre(last, pid)
                t_vuelta = tiempo_entre(pid, 18)
                t_extra = t_viaje + tiempo_descarga + t_vuelta
                if tiempo_total + tiempo_ruta + t_extra > jornada_min:
                    continue

                entrega_nodo = {}
                total_nodo = 0
                for esp in demanda_poligonos:
                    disp = disponibles[esp][dia]
                    dem = demanda_restante[esp].get(pid, 0)
                    cap_rest = capacidad_camion - carga - total_nodo
                    q = min(disp, dem, cap_rest)
                    if q > 0:
                        entrega_nodo[esp] = q
                        total_nodo += q

                if total_nodo > 0:
                    ruta.append(pid)
                    tiempo_ruta += t_viaje + tiempo_descarga
                    for esp, q in entrega_nodo.items():
                        disponibles[esp][dia] -= q
                        demanda_restante[esp][pid] -= q
                        entregado[esp] += q
                    carga += total_nodo
                    detalle.append((pid, entrega_nodo))
                    encontrado = True
                    break

            if not encontrado:
                break

        if len(ruta) > 1:
            tiempo_ruta += tiempo_entre(ruta[-1], 18)
            tiempo_total += tiempo_ruta
            rutas_dia.append({
                "Día": dia,
                "Ruta": " → ".join(map(str, ruta + [18])),
                "Duración_min": round(tiempo_ruta),
                "Unidades": carga,
                "Detalle": detalle
            })
        else:
            break

    return rutas_dia, entregado

def procesar_entregas(inventario, entregas, entregado, dia):
    for esp, cantidad in entregado.items():
        if cantidad > 0:
            inventario[esp][dia] -= cantidad
            entregas.append({
                "Especie": esp,
                "Día entrega": dia,
                "Cantidad": cantidad,
                "Costo plantación": cantidad * costo_plantacion
            })

def realizar_compras(inventario, compras, oferta_usada, dia):
    proveedores_hoja = set()
    for esp in demanda_poligonos:
        if dia + aclimatacion_min_dias >= dias_totales:
            continue

        total_dem = sum(demanda_poligonos[esp].values())
        total_cmp = sum(c["Cantidad"] for c in compras if c["Especie"] == esp)
        restante = total_dem - total_cmp

        espacio_disp = espacio_max_almacen - inventario[esp][dia]
        max_posible = min(restante, espacio_disp)
        if max_posible <= 0:
            continue

        opciones = []
        for p, datos in demandas_oferta.items():
            info = datos.get(esp)
            if info:
                disponible_prov = info["max_oferta"] - oferta_usada[p][esp]
                if disponible_prov > 0:
                    opciones.append((p, info["costo"], disponible_prov))
        opciones.sort(key=lambda x: x[1])

        restante_a_comprar = max_posible
        for p_sel, costo, dispo in opciones:
            if restante_a_comprar <= 0:
                break
            cantidad = min(dispo, restante_a_comprar)
            costo_trans = costo_transporte if p_sel not in proveedores_hoja else 0

            compras.append({
                "Especie": esp,
                "Día pedido": dia,
                "Proveedor": p_sel,
                "Cantidad": cantidad,
                "Costo compra": cantidad * costo,
                "Costo transporte": costo_trans
            })
            proveedores_hoja.add(p_sel)
            oferta_usada[p_sel][esp] += cantidad
            inventario[esp][dia + 1] += cantidad

            restante_a_comprar -= cantidad
            espacio_disp -= cantidad

def simular():
    inventario = {esp: [0] * (dias_totales + 1) for esp in demanda_poligonos}
    demanda_restante = {esp: demanda_poligonos[esp].copy() for esp in demanda_poligonos}
    ofertas_usadas = {p: {esp: 0 for esp in demanda_poligonos} for p in demandas_oferta}
    compras = []
    entregas = []
    rutas = []

    # Lista para capturar inventario diario
    lista_inventario = []

    for dia in range(dias_totales):
        actualizar_inventario(inventario, dia)

        # Capturar inventario del día para el DataFrame
        row_inv = {"Día": dia}
        for esp in inventario:
            row_inv[esp] = inventario[esp][dia]
        lista_inventario.append(row_inv)

        disponibles = calcular_disponibles(inventario)
        rutas_dia, entregado = planificar_rutas(dia, disponibles, demanda_restante)
        rutas.extend(rutas_dia)
        procesar_entregas(inventario, entregas, entregado, dia)
        realizar_compras(inventario, compras, ofertas_usadas, dia)

    # Capturar inventario del último día
    row_inv = {"Día": dias_totales}
    for esp in inventario:
        row_inv[esp] = inventario[esp][dias_totales]
    lista_inventario.append(row_inv)

    # Convertir lista de inventario a DataFrame y mostrar
    df_inventario = pd.DataFrame(lista_inventario)
    print("=== INVENTARIO DIARIO ===")
    print(df_inventario)

    return pd.DataFrame(compras), pd.DataFrame(entregas), pd.DataFrame(rutas)

if __name__ == '__main__':
    df_compras, df_entregas, df_rutas = simular()
    print("\n=== COMPRAS ===\n", df_compras)
    print("\n=== ENTREGAS ===\n", df_entregas)
    print("\n=== RUTAS ===\n", df_rutas)
